In [ ]:
!pip install tensorflow_hub tensorflow xgboost scikit-learn pandas numpy

In [ ]:
!git clone https://github.com/SenticNet/stress-detection

Cloning into 'stress-detection'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 2.88 MiB | 15.59 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
import pandas as pd
# Load datasets
df = pd.read_excel("/content/stress-detection/Reddit_Title.xlsx")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

def preprocess_data(df):
    df = df.dropna()
    X = df['title'].astype(str).tolist()
    y = df['label'].astype(int).values
    return X, y

X, y = preprocess_data(df)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

def elmo_embedding(texts):
    return elmo.signatures['default'](tf.constant(texts))['elmo'].numpy().mean(axis=1)

# Generate ELMo embeddings
X_train_elmo = np.array([elmo_embedding([text])[0] for text in X_train])
X_test_elmo = np.array([elmo_embedding([text])[0] for text in X_test])

# Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_elmo, y_train)
y_pred_log = log_reg.predict(X_test_elmo)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_log))

# SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_elmo, y_train)
y_pred_svm = svm_model.predict(X_test_elmo)
print("SVM:")
print(classification_report(y_test, y_pred_svm))

# XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_elmo, y_train)
y_pred_xgb = xgb_model.predict(X_test_elmo)
print("XGBoost:")
print(classification_report(y_test, y_pred_xgb))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       556
           1       0.94      0.92      0.93       556

    accuracy                           0.93      1112
   macro avg       0.93      0.93      0.93      1112
weighted avg       0.93      0.93      0.93      1112

SVM:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       556
           1       0.93      0.91      0.92       556

    accuracy                           0.92      1112
   macro avg       0.92      0.92      0.92      1112
weighted avg       0.92      0.92      0.92      1112



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:49:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       556
           1       0.93      0.91      0.92       556

    accuracy                           0.92      1112
   macro avg       0.92      0.92      0.92      1112
weighted avg       0.92      0.92      0.92      1112

